In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models import model_from_checkpoint
from utils.tokenizer import BPETokenizer, BOS_ID, EOS_ID, PAD_ID

model = model_from_checkpoint("fix-early-eos/epoch_5.pt")
tokenizer = BPETokenizer.load("py150k_large")

In [3]:
from utils.dataset import MemmapDataset


ds = MemmapDataset("train", "py150k_large")
tokenizer.detokenize(ds[3].tolist())

'if len(parts) > 2:\n            identity = parts[0:-2]\n            blob = parts[-1]\n        elif len(parts) == 2:\n            identity = parts[0:-1]\n            blob = parts[-1]\n        else:\n            identity = None\n            blob = parts[0]\n        event = Event.unpack(get_pyzmq_frame_buffer(blob))\n        event.identity = identity\n        if self._debug:\n            logger.debug(\'<-- %s\', event)\n        return event\n\n    def setsockopt(self, *args):\n        return self._socket.setsockopt(*args)\n\n    @property\n    def context(self):\n        return self._context\n#!/usr/bin/env python\n"""Django\'s command line utility."""\n\nimport os\nimport sys\n\nif __name__ == "__main__":\n    os.environ.setdefault("DJANGO_SETTINGS_MODULE", "project.settings")\n\n    from django.core.management import execute_from_command_line\n\n    execute_from_command_line(sys.argv)\n"""Installer for hippybot\n"""\n\nimport os\ncwd = os.path.dirname(__file__)\n__version__ = open(os.p

In [4]:
import torch
from torch.utils.data import ConcatDataset, DataLoader, random_split

def collate_fn(batch:list[torch.Tensor], max_len:int=2048):
    batch = [x[:max_len] for x in batch]
    batch = [
        torch.cat([torch.tensor([BOS_ID]), x, torch.tensor([EOS_ID])])
        for x in batch
    ]
    return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=PAD_ID)

train_ds = MemmapDataset("train", "py150k_large")
val_ds = MemmapDataset("eval", "py150k_large")
train_extra_ds, val_ds, _ = random_split(val_ds, [0.85, 0.1, 0.05])
train_ds = ConcatDataset([train_ds, train_extra_ds]) # 142.5k instead of 100k

train_dl = DataLoader(train_ds, batch_size=32, collate_fn=collate_fn, prefetch_factor=4, num_workers=4, persistent_workers=True)
val_dl = DataLoader(val_ds, batch_size=32, collate_fn=collate_fn, prefetch_factor=4, num_workers=4, persistent_workers=True)


In [5]:
batch = next(iter(train_dl))

In [6]:
batch[:,-1]

tensor([1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
tokenizer.detokenize(batch[0].tolist())

'<bos># -*- coding: utf-8 -*-\n\n""" Provides ``mapping`` of url paths to request handlers.\n"""\n\nfrom bootstrap import Bootstrap\nfrom fund import InstantPaymentNotificationHandler\nfrom fund import ThankYouHandler\nfrom view import *\n\nmapping = [(\n        r\'/\',\n        Index\n    ), (\n        r\'/ipn\',\n        InstantPaymentNotificationHandler\n    ), (\n        r\'/thank-you\',\n        ThankYouHandler\n    ), (\n        r\'/about\\/?\',\n        About\n    ), (\n        r\'/guide\\/?\',\n        Guide\n    ), (\n        r\'/guide/download\\/?\',\n        Download\n    ), (\n        r\'/guide/standards\\/?\',\n        Standards\n    ), (\n        r\'/community\\/?\',\n        Community\n    ), (\n        r\'/news\\/?\',\n        News\n    ), (\n        r\'/support\\/?\',\n        Support\n    ), (\n        r\'/contact\\/?\',\n        Contact\n    ), (\n        r\'/press\\/?\',\n        Press\n    ), (\n        r\'/legal/terms\',\n        Terms\n    ), (\n        r\'/libra

In [8]:
samples = model.generate(4, 100, nucleus_threshold=0.85)
programs = [tokenizer.detokenize(tokens) for tokens in samples]

for program in programs[:10]:
    print(tokenizer.color_text_ansi(program))
    print()

self.assertEqual(property, (tp[0], f.mgg_device, other.__main__)
        # so number of outputs are conditions with namespace to you can be only active method for u '

import log
import plot
def remove_response(data)

def test_

   self.param merge_scales_arruides: "
CreateSettings:<eos>: """
	"""
This class is managed to a list of bitgy.
        @just called by application region
  certings.  In the operators of the documentation."""
        

>>> ColorFilter(managements, attrs=None, label=msg):
            return False

# If the remove the computed. Django sentication totalp the keys. Asservice an AN IP BASIE, EXENEDECUDIRE: DIT

if k == 40 # src. No Is the message pbradient number of ranger this validation.

    PROPER ALUE THS EOF PROWARRANTIENTIES OARL, WARES OR OF T



In [9]:
for sample in samples:
    print(sample)

[206, 533, 15, 319, 400, 283, 183, 15, 91, 87, 547, 68, 183, 77, 21, 84, 78, 78, 70, 189, 323, 230, 183, 86, 575, 89, 21, 286, 259, 184, 286, 231, 239, 393, 7, 85, 312, 73, 518, 351, 484, 87, 306, 197, 205, 193, 251, 249, 221, 197, 417, 7, 267, 376, 551, 193, 305, 96, 503, 7, 74, 499, 550, 186, 83, 273, 551, 202, 383, 7, 209, 195, 86, 218, 300, 92, 7, 14, 246, 304, 444, 5, 304, 87, 241, 91, 5, 250, 185, 260, 383, 70, 530, 554, 15, 348, 226, 5, 250, 457]
[422, 206, 320, 342, 7, 209, 89, 222, 70, 432, 200, 552, 70, 205, 89, 92, 280, 552, 215, 9, 5, 42, 185, 237, 58, 536, 202, 85, 78, 90, 33, 1, 215, 277, 5, 4, 277, 5, 466, 276, 474, 276, 259, 85, 504, 76, 218, 305, 301, 431, 7, 351, 73, 414, 78, 96, 21, 181, 39, 81, 569, 198, 541, 208, 218, 73, 273, 405, 562, 386, 7, 185, 78, 80, 186, 5, 178, 230, 89, 202, 85, 78, 216, 178, 441, 460, 296, 199, 213, 188, 197, 351, 243, 340, 74, 312, 291, 386, 21, 549]
[37, 37, 37, 7, 42, 335, 188, 398, 83, 275, 15, 259, 85, 504, 76, 524, 378, 213, 307, 90

In [24]:
code = """from django.views.generic import ListView
from books.models import Publisher


class PublisherListView(ListView):
    model = Publisher
    context_object_name = "my_favorite_publishers"
"""

tokens = torch.tensor(tokenizer.tokenize(code)).unsqueeze(0)

pred = model.generate(1, max_len=2048, starting_tokens=tokens, nucleus_threshold=0.5)

print(tokenizer.color_text_ansi(tokenizer.detokenize(pred[0])))

from django.views.generic import ListView
from books.models import Publisher


class PublisherListView(ListView):
    model = Publisher
    context_object_name = "my_favorite_publishers"
					tuple_name = _("index 'SubB', None))
        ]

            if number in dict() == 0:
                        # Python a message if there is remove method by applicable law or agreed to in writing, which record and limitations under the License.
# You may obtain a copy of the License at
#
#         http://www.apache.org/licenses/LICENSES FOR ENT CODED ANY KIND, either express or implied. The SOFT LICESS COPY BUTHOUT SALIE THE COLICENSE, OR OF WARRANTIES OR ANY KIND, either express or implied.
# See the License at
#
#         http://www.apache.org/licenses/LICENSE-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
# A BE DITIONS OF THE ANY THE IS IF DATED WARRANTIES OR CONTRICATIONS OR CONDITIONS OF ANY KIND, eith

In [11]:
pred[0]

[339,
 498,
 323,
 76,
 94,
 216,
 222,
 384,
 247,
 74,
 7,
 304,
 51,
 559,
 61,
 80,
 76,
 94,
 488,
 428,
 86,
 82,
 216,
 381,
 197,
 304,
 55,
 418,
 236,
 354,
 199,
 246,
 5,
 474,
 55,
 418,
 236,
 354,
 199,
 51,
 559,
 61,
 80,
 76,
 94,
 15,
 51,
 559,
 61,
 80,
 76,
 94,
 387,
 381,
 210,
 55,
 418,
 236,
 354,
 199,
 192,
 251,
 439,
 229,
 448,
 229,
 267,
 210,
 9,
 84,
 437,
 344,
 93,
 188,
 539,
 70,
 87,
 418,
 236,
 354,
 199,
 90,
 9,
 5,
 333,
 4,
 248,
 360,
 257,
 559,
 233,
 70,
 280,
 321,
 290,
 206,
 392,
 363,
 15,
 317,
 248,
 206,
 77,
 253,
 399,
 7,
 276,
 332,
 321,
 206,
 185,
 78,
 559,
 199,
 70,
 280,
 15,
 74,
 551,
 397,
 70,
 280,
 36,
 206,
 319,
 358,
 229,
 280,
 183,
 206,
 267,
 183,
 185,
 385,
 89,
 91,
 183,
 9,
 87,
 308,
 91,
 449,
 9,
 215,
 9,
 452,
 91,
 87,
 33,
 528,
 94,
 94,
 94,
 21,
 311,
 551,
 397,
 21,
 188,
 78,
 22,
 562,
 201,
 182,
 90,
 22,
 51,
 48,
 42,
 44,
 53,
 58,
 44,
 20,
 25,
 21,
 23,
 478,
 402,
 60,
 85,
 